In [1]:
def fast_exp(base, exponent, modulus):
  result = 1
  while exponent != 0:
    if exponent % 2 == 1:
      result = (result * base) % modulus
    base = base**2 % modulus
    exponent = exponent // 2
  return result

def extended_euclidean(m, n):
  if n == 0:
    return 1, 0, m
  x, y, g = extended_euclidean(n, m % n)
  return y, x - (m // n)*y, g

def inv_mod(a, modulus):
  s, _, g = extended_euclidean(a, modulus)
  assert g == 1, ValueError('the modular inverse does not exist')
  return s % modulus

def crt(remainders, moduli):
  assert len(remainders) == len(moduli), ValueError('the lists remainders and moduli are not the same length')
  assert len(remainders) > 0, ValueError('the list lengths must be greater than one')

  first_modulus = moduli[0]
  first_remainder = remainders[0]

  if len(remainders) == 1:
    return first_remainder % first_modulus, first_modulus
  
  consecutive_remainder, consecutive_modulus = crt(remainders[1:], moduli[1:])
  u, v, g = extended_euclidean(consecutive_modulus, first_modulus)

  assert g == 1, ValueError('the moduli are not relatively prime')

  return (first_remainder*consecutive_modulus*u + consecutive_remainder*first_modulus*v) % (first_modulus*consecutive_modulus), first_modulus*consecutive_modulus

def polynomial_root(exponent, a, modulus):
  u = inv_mod(exponent, modulus - 1)

  result = fast_exp(a, u, modulus)

  return result